# Objective
In this file, we are trying to create an agentic ai solution with custom optimization function. First, there is one agent which will analyze some percentile values as highest and lowest ranges from a 24 hours distribution and then pass the maximum and minimum value to another agent which will generate the maximum value of the  objective function.

In [1]:
!pip install langchain_community --quiet
!pip install langchain_groq --quiet
!pip install langchain-google-genai --quiet
!pip install langchain-openai --quiet
!apt-get install -y -qq coinor-cbc --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 69.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.1/131.1 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.8/47.8 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 22.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.18 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.6 kB 5.5 MB/s eta 0:00:00
Selecting previously unselected package coinor-libcoinutils3v5:amd64.
(Reading database ... 126284 files and directories currently in

In [3]:
from langchain.prompts import PromptTemplate
from langchain_core.tools import tool
import langchain_community
import warnings
warnings.filterwarnings('ignore')
import math
import os
from geopy.geocoders import Nominatim
from langchain_core.messages import AIMessage,HumanMessage
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from google.colab import userdata
from langchain.agents import tool
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI
from pyomo.environ import *
import pyomo
import random
import numpy as np

In [4]:
# List of functions

@tool
def get_dict_values(input_dict:dict, high_percentile:int, low_percentile:int=10):
  '''
  This function takes dictionary as input and return defined percentile ranges value of a dictionary
  '''
  # Extract the values from the dictionary
  values = list(input_dict.values())

  # Calculate the 75th percentile
  high_val = np.percentile(values, high_percentile)
  low_val = np.percentile(values, low_percentile)

  return (high_val, low_val)

@tool
def sample_optimization(high_val:float, low_val:float):
  '''
  This is an optimization function which will take two integer inputs, perform the necessary
  calculations and provide float output
  '''
  model = ConcreteModel()

  # Define variables
  model.x = Var(within=NonNegativeReals)
  model.y = Var(within=NonNegativeReals)

  # Define objective function: Minimize x + y
  model.objective = Objective(expr=model.x + model.y, sense=minimize)

  # Define constraints
  model.constraint1 = Constraint(expr=model.x + 2*model.y >= int(high_val))
  model.constraint2 = Constraint(expr=model.x - model.y <= int(low_val))

  # Solve the model using GLPK solver
  solver = pyomo.environ.SolverFactory('cbc')
  # solver = pyomo.environ.SolverFactory('gurobi')
  solver.solve(model)

  return (value(model.objective))

In [5]:
# input dictionary
input_dict = {}
random.seed(33)
for i in range(1,25):
  input_dict['hour_'+str(i)]= random.randint(23,79)

In [14]:
prompt_openai = ChatPromptTemplate.from_messages(
    [
        ("system", "You are an agentic AI solution. You have 2 functions which you need to execute as per human input prompt. The first function is 'get_dict_values_wrapper' which takes two integers: high percentile and low percentile. Use the dictionary already provided in the environment. For valid integer percentiles, call get_dict_values_wrapper and generate high_val and low_val parameters. Use these as inputs to 'sample_optimization' function and produce final output."),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

prompt_gemini = ChatPromptTemplate.from_messages(
    [
        ("system", "You are an agentic AI solution. You have 2 functions which you need to execute as per human input prompt. The first function is 'get_dict_values' function which will take an input dictionary and then two integer values from human prompt, which are called high percentile value and low percentile value. Use the dictionary provided in the 'input_dict' variable for this function. In case any of the two numbers are input anything other than integer, please return a message as 'You should provide only integer values as high percentile and low percentile values. Please modify the prompt' and complete the chain. For valid values of high percentile and low percentile, execute 'get_dict_values' and generate high_val and low_val parameters. Use these parameters as input to 'sample_optimization' function and generate the final output."),
        ("human", "{input}\n\nHere is the dictionary to use:\n{input_dict}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

## **Test run for OpenAI Agent Information passdown**

### For OpenAI agentic approach with Lngchain, the tool definitation will change. We shall prepare a wrapper to pass the pre-defined data and then call the wrapper function during human prompting

In [17]:
def get_dict_values(input_dict:dict, high_percentile:int, low_percentile:int):
      '''
      This function takes dictionary as input and return defined percentile ranges value of a dictionary
      '''
      # Extract the values from the dictionary
      values = list(input_dict.values())

      # Calculate the 75th percentile
      high_val = np.percentile(values, high_percentile)
      low_val = np.percentile(values, low_percentile)

      return (high_val, low_val)

@tool
def get_dict_values_wrapper(high_percentile: int, low_percentile: int):
    '''
    This is a wrapper function to embed input dictionary into the function directly and need not
    to provide through the prompt itself
    '''
    return get_dict_values(input_dict, high_percentile, low_percentile)

@tool
def sample_optimization(high_val:float, low_val:float):
      '''
      This is an optimization function which will take two float inputs, perform the necessary
      calculations and provide float output
      '''
      model = ConcreteModel()

      # Define variables
      model.x = Var(within=NonNegativeReals)
      model.y = Var(within=NonNegativeReals)

      # Define objective function: Minimize x + y
      model.objective = Objective(expr=model.x + model.y, sense=minimize)

      # Define constraints
      model.constraint1 = Constraint(expr=model.x + 2*model.y >= int(high_val))
      model.constraint2 = Constraint(expr=model.x - model.y <= int(low_val))

      # Solve the model using GLPK solver
      solver = pyomo.environ.SolverFactory('cbc')
      # solver = pyomo.environ.SolverFactory('gurobi')
      solver.solve(model)

      return (value(model.objective))

In [18]:
# Model setting
os.environ["OPENAI_API_KEY"] = userdata.get('OPEN_AI_TOKEN')
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
tools = [get_dict_values_wrapper, sample_optimization]
agent = create_tool_calling_agent(llm, tools, prompt_openai)

# run
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
message = '''
Calculate the percentil values using 'get_dict_values' function. Consider 85 as high percentile
and 10 as low percentile value. Pass the output to sample_optimization function and
return the final output"
'''
result = agent_executor.invoke({"input": message})


print(result["output"])



> Entering new AgentExecutor chain...

Invoking: `get_dict_values_wrapper` with `{'high_percentile': 85, 'low_percentile': 10}`


(np.float64(66.0), np.float64(37.9))
Invoking: `sample_optimization` with `{'high_val': 66, 'low_val': 37.9}`


33.0The final output from the optimization process is **33.0**.

> Finished chain.
The final output from the optimization process is **33.0**.


## **Test run for Gemini Agent Information passdown**

In [8]:
# Model setting
os.environ["GOOGLE_API_KEY"] = userdata.get('GEMINI_TOKEN')
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")
tools = [get_dict_values, sample_optimization]
agent = create_tool_calling_agent(llm, tools, prompt_gemini)

# run
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
message = "Calculate the percentil values of the provided dictionary using 'get_dict_values' function. Consider 85 as high percentile and 10 as low percentile value. Pass the output to sample_optimization function and return the final output"
result = agent_executor.invoke({"input":  message, "input_dict": input_dict})
print(result["output"])



> Entering new AgentExecutor chain...

Invoking: `get_dict_values` with `{'low_percentile': 10.0, 'input_dict': {'hour_5': 75.0, 'hour_17': 43.0, 'hour_22': 41.0, 'hour_12': 56.0, 'hour_18': 66.0, 'hour_21': 51.0, 'hour_11': 57.0, 'hour_13': 34.0, 'hour_3': 63.0, 'hour_16': 55.0, 'hour_14': 64.0, 'hour_2': 33.0, 'hour_1': 59.0, 'hour_23': 66.0, 'hour_19': 56.0, 'hour_9': 64.0, 'hour_7': 40.0, 'hour_4': 37.0, 'hour_10': 79.0, 'hour_24': 63.0, 'hour_20': 53.0, 'hour_6': 76.0, 'hour_8': 53.0, 'hour_15': 62.0}, 'high_percentile': 85.0}`


(np.float64(66.0), np.float64(37.9))
Invoking: `sample_optimization` with `{'low_val': 37.9, 'high_val': 66.0}`


33.0The optimization function returned 33.

> Finished chain.
The optimization function returned 33.
